# Laptop Price prediction

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
import seaborn as sns
import numpy as np
import matplotlib.ticker as mtick

In [1]:
df = pd.read_csv('../input/laptop-price/laptop_price.csv',encoding = "ISO-8859-1")

In [1]:
df.info()

# Featuring Engineering

In [1]:
df.select_dtypes(exclude=['float64','int64']).describe()

In [1]:
df = df.drop('Product', axis=1)
df

In [1]:
df['Weight'] = df['Weight'].str.extract(r'(\d.\d+|\d.\d|\d)')

In [1]:
df['Weight'] = df['Weight'].astype('float64')

In [1]:
df.Cpu.unique()

In [1]:
df['Cpu_model'] = df['Cpu'].str.extract(r'(i\d|AMD|Samsung|Pentium|Celeron|Atom|Core M|Xeon)')

In [1]:
df.Cpu_model.isnull().sum()

In [1]:
df['Cpu_freq'] = df['Cpu'].str.extract(r'(\d+\.\d+GHz|\d+GHz)')

In [1]:
df['Cpu_freq'] = df['Cpu_freq'].str.replace('GHz','').astype('float64')

In [1]:
df['Cpu_freq'].isnull().sum()

In [1]:
df.ScreenResolution.unique()

In [1]:
df['Resolution'] = df.ScreenResolution.str.extract(r'(\d+x\d+)')

In [1]:
df.Resolution.isnull().sum()

In [1]:
df.Memory.unique()

In [1]:
df['Memory_Size'] = df.Memory.str.extract(r'(\d+GB|\d+TB)')

In [1]:
df['Memory_Size'] = df.Memory_Size.str.extract(r'(\d+)').astype('int64')

In [1]:
df['Memory_Size'] = list((map((lambda x: x*1000 if x < 16 else x),df['Memory_Size'])))

In [1]:
df['Storage_type'] = df.Memory.str.extract(r'(.+[+].+|SSD|HDD|Flash Storage|Hybrid)')

In [1]:
df['Storage_type'].isnull().sum()

In [1]:
df.Gpu.unique()

In [1]:
df['Gpu_brand'] = df.Gpu.str.extract(r'(Nvidia|AMD|Intel)')

In [1]:
print([col for col in df.columns if df[col].isnull().sum() > 0])

In [1]:
df.Ram.unique()

In [1]:
df['Ram'] = df.Ram.str.extract(r'(\d+)').astype('int64')

In [1]:
df.info()

In [1]:
df1 = df.drop(['ScreenResolution','Cpu','Memory','Gpu','laptop_ID'], axis= 1)

In [1]:
df1.select_dtypes(exclude=['float64','int64']).describe()

In [1]:
df1.dropna(inplace=True)

In [1]:
df1.columns

# Exploratory data analisys

## Which company has higher sales?

In [1]:
company_sales = df1.groupby('Company').sum().sort_values(by = 'Price_euros', ascending = False)
company_sales
plt.figure(figsize=(15,5))
plt.xticks(rotation=90)
sns.barplot(data = company_sales, x= company_sales.index , y= 'Price_euros')

## How many Ram memory customers prefer buying a notebook?

In [1]:
ram_sales = df1.groupby('Ram').count().sort_values(by = 'Company', ascending = True)
sales_pct = list(map(lambda x: (x/1302)*100, ram_sales.Company ))

fig4, ax4 = plt.subplots(figsize=(10,8))
ax4.pie(sales_pct, startangle= 90, labels = list(map(lambda x: f'{x:.1f}%',sales_pct)), rotatelabels= True)
ax4.legend(list(map(lambda x: f'{x} Mb RAM',ram_sales.index)), loc = 'best')
plt.show()


## Which OS is preferred ?

In [1]:
os_sales = df1.groupby('OpSys').count().sort_values(by = 'Company', ascending = False)
pct = list(map(lambda x: (x / 1302)*100, os_sales.Company))

fig, ax1 = plt.subplots(figsize=(10,5))
sns.barplot(data = os_sales, x= os_sales.index, y= pct)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter())

plt.show()

## Which Cpu frequency customers choose?

In [1]:
freq_sales = df1.groupby('Cpu_freq').count().sort_values(by = 'Company', ascending = False)

freq_pct = list(map(lambda x: (x/1302)*100, freq_sales.Company))

fig2, ax2 = plt.subplots(figsize = (10,5))
sns.barplot(x = freq_sales.index, y = freq_pct)
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

## Which resolution do the customers prefer?

In [1]:
resolution_sales = df1.groupby(by = 'Resolution').count().sort_values(by = 'Company', ascending = False)

res_pct = list(map(lambda x: (x/1302)*100, resolution_sales.Company))

fig, ax3 = plt.subplots(figsize = (10,5))
sns.barplot(data = resolution_sales, x = resolution_sales.index, y = res_pct)
ax3.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.xticks(rotation = 90)
plt.show()

## Which variables are correlated to the price?

In [1]:
fig, ax = plt.subplots(figsize = (10,5))
sns.heatmap(df1.corr(), annot = True)
plt.show()

In [1]:
df1.info()

# Preprocessing and Modeling

In [1]:
x = df1.drop('Price_euros', axis=1)
y = df1.Price_euros

In [1]:
x_train, x_val, y_train, y_val = train_test_split(x, y)

In [1]:
good_cat_cols = [col for col in x_train.columns if x_train[col].nunique() < 10 and x_train[col].dtype == 'object']
bad_cat_cols = [col for col in x_train.columns if x_train[col].nunique() > 10 and x_train[col].dtype == 'object']
num_cols = [col for col in x_train.columns if x_train[col].dtype != 'object']

In [1]:
cat_transf = OneHotEncoder(handle_unknown='ignore', sparse = False)
num_transf = MinMaxScaler(feature_range= (-1, 1))

In [1]:
preprocessor = ColumnTransformer(transformers = [('num', num_transf, num_cols),('cat', cat_transf, good_cat_cols)])

In [1]:
num_cols

## XGboost model

In [1]:
model = XGBRegressor(n_estimators = 10)

In [1]:
clf = Pipeline(steps= [('preprocessor',preprocessor),('model', model)])

In [1]:
clf.fit(x_train, y_train)

In [1]:
preds = clf.predict(x_val)

In [1]:
xgb_r2 = r2_score(y_val, preds)

In [1]:
mean_absolute_error(y_val, preds)

## Random Forest model

In [1]:
model1 = RandomForestRegressor(n_estimators = 10)

In [1]:
clf2 = Pipeline(steps=[('preprocessor',preprocessor),('model1', model1)])

In [1]:
clf2.fit(x_train, y_train)

In [1]:
preds2 = clf2.predict(x_val)

In [1]:
rf_r2 = r2_score(y_val, preds2)

In [1]:
print(f'R2 for Random forest regression = {rf_r2:.2f}\nR2 for XGBoost Regressor = {xgb_r2:.2f}')

In [1]:
cat_features = preprocessor.transformers_[1][1].get_feature_names().tolist()

In [1]:
feature_names = num_cols

In [1]:
feature_names.extend(cat_features)

## What is the most important feature for price?

In [1]:
plt.figure(figsize = (10,5))
sns.barplot(x = feature_names ,y= model.feature_importances_)
plt.xticks(rotation = 90)
plt.title('Feature importance')
plt.show()